In [1]:
import torch
import time
import numpy as np

import sigkernel

import matplotlib.pyplot as plt

In [32]:
dyadic_order = 5
_naive_solver = False

In [33]:
# specify static kernel
static_kernel = sigkernel.LinearKernel()
# static_kernel = sigkernel.RBFKernel(sigma=.5)

# initialize signature kernel
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order, _naive_solver)

# Sig Loss gradients

In [4]:
A = 1
M = 3
N = 2
D = 2

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(A,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

In [8]:
X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

X_gpu = X_naive.clone().cuda()
Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
X_gpu.requires_grad = True

In [9]:
t = time.time()
l_naive = sigkernel.SigLoss_naive(static_kernel, dyadic_order, _naive_solver).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(l_naive)

time: 0.021 s
tensor(54.1607, dtype=torch.float64, grad_fn=<SubBackward0>)


In [10]:
t = time.time()
l_cpu = signature_kernel.compute_distance(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(l_cpu)

time: 0.002 s
tensor(54.1607, dtype=torch.float64, grad_fn=<SubBackward0>)


In [12]:
t = time.time()
l_gpu = signature_kernel.compute_distance(X_gpu,Y_gpu)
print('time:', np.round(time.time()-t,3), 's')
print(l_gpu)

time: 0.004 s
tensor(54.1607, device='cuda:0', dtype=torch.float64, grad_fn=<SubBackward0>)


In [13]:
t = time.time()
l_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.078 s


In [14]:
t = time.time()
l_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

201
215
220
250
259
265
273
278
285
292
201
215
220
250
259
265
273
278
285
292
time: 0.052 s


In [15]:
t = time.time()
l_gpu.backward()
print('time:', np.round(time.time()-t,3), 's')

201
215
220
239
250
259
265
273
278
285
292
201
215
220
239
250
259
265
273
278
285
292
time: 0.633 s


In [16]:
X_naive.grad

tensor([[[ 3.7926,  0.2543],
         [ 0.0415,  0.2326],
         [-3.8342, -0.4869]]], dtype=torch.float64)

In [17]:
X_cpu.grad

tensor([[[ 4.5570,  0.6105],
         [ 0.1850,  0.2697],
         [-4.7420, -0.8801]]], dtype=torch.float64)

In [18]:
X_gpu.grad.cpu()

tensor([[[ 4.5570,  0.6105],
         [ 0.1850,  0.2697],
         [-4.7420, -0.8801]]], dtype=torch.float64)

# Sig MMD gradients

In [34]:
A = 2
B = 3
M = 4
N = 3
D = 2

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(B,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

In [35]:
X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

X_gpu = X_naive.clone().cuda()
Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
X_gpu.requires_grad = True

In [36]:
t = time.time()
mmd_naive = sigkernel.SigMMD_naive(static_kernel, dyadic_order, _naive_solver).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_naive)

time: 3.351 s
tensor(0.3812, dtype=torch.float64, grad_fn=<SubBackward0>)


In [37]:
t = time.time()
mmd_cpu = signature_kernel.compute_mmd(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_cpu)

time: 0.008 s
tensor(0.3812, dtype=torch.float64, grad_fn=<SubBackward0>)


In [38]:
t = time.time()
mmd_gpu = signature_kernel.compute_mmd(X_gpu,Y_gpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_gpu)

time: 0.005 s
tensor(0.3812, device='cuda:0', dtype=torch.float64, grad_fn=<SubBackward0>)


In [39]:
t = time.time()
mmd_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 11.47 s


In [40]:
t = time.time()
mmd_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.014 s


In [41]:
t = time.time()
mmd_gpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.01 s


In [42]:
X_naive.grad

tensor([[[ 2.8969e-01,  5.5979e-01],
         [ 2.0302e-02, -1.8834e-02],
         [ 2.9239e-02, -2.2273e-02],
         [-3.3923e-01, -5.1868e-01]],

        [[ 3.2008e-01,  5.5694e-01],
         [ 3.7343e-02, -2.0445e-02],
         [-5.1021e-04, -1.6423e-02],
         [-3.5692e-01, -5.2008e-01]]], dtype=torch.float64)

In [43]:
X_cpu.grad

tensor([[[ 2.8698e-01,  5.5858e-01],
         [ 1.9109e-02, -2.0256e-02],
         [ 3.0031e-02, -2.0848e-02],
         [-3.3612e-01, -5.1748e-01]],

        [[ 3.1639e-01,  5.5519e-01],
         [ 3.7546e-02, -2.1233e-02],
         [ 2.5978e-04, -1.5088e-02],
         [-3.5420e-01, -5.1887e-01]]], dtype=torch.float64)

In [44]:
X_gpu.grad.cpu()

tensor([[[ 2.8698e-01,  5.5858e-01],
         [ 1.9109e-02, -2.0256e-02],
         [ 3.0031e-02, -2.0848e-02],
         [-3.3612e-01, -5.1748e-01]],

        [[ 3.1639e-01,  5.5519e-01],
         [ 3.7546e-02, -2.1233e-02],
         [ 2.5978e-04, -1.5088e-02],
         [-3.5420e-01, -5.1887e-01]]], dtype=torch.float64)